# Example Optimiser

In [11]:
import os 
import sys
import json
sys.path.append(os.path.abspath(".."))
from velopix_wrappers.optimizers import BaseOptimizer
from velopix_wrappers.velopix_pipeline import TrackFollowingPipeline, GraphDFSPipeline, SearchByTripletTriePipeline
from typing import Any, Dict
import numpy as np
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

## Implement the optimiser child class

In [12]:
from typing import Any, Dict, Literal, List
import random
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from velopix_wrappers.optimizers import BaseOptimizer, pMap

class Bayesian(BaseOptimizer):
    def __init__(
        self,
        learning_rate: float,
        max_iterations: int = 100,
        target_score: float = 0.3,
        objective: Literal["min", "max"] = "min"
    ):
        super().__init__(objective=objective)
        self.learning_rate = learning_rate
        self.max_iterations = max_iterations
        self.target_score = target_score
        self.current_iteration = 0

        # Storage for GP model
        self.X: List[List[Any]] = []
        self.Y: List[float] = []

        # Gaussian Process model
        kernel = C(1.0, (1e-4, 1e1)) * RBF(1.0, (1e-4, 1e1))
        self.gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
        
        # State tracking
        self._stopped = False
        self._current_config = None

    def init(self) -> pMap:
        """
        Initializes with a random point within bounds.
        """
        self._stopped = False
        self.current_iteration = 0
        self.X = []
        self.Y = []
        
        # Sample initial point
        initial_config = self._sample_random_point()
        
        # Store for evaluation in first call to next()
        self._current_config = initial_config
        
        return initial_config

    def next(self) -> pMap:
        """
        Evaluates the current configuration and returns a new one.
        """
        # Evaluate the current configuration (from previous init/next call)
        score = self.objective_func([1.0, 1.0, 1.0, -10.0])
        
        # Record evaluation
        if self._current_config:
            self.X.append(list(self._current_config.values()))
            self.Y.append(score)
            
            # Update best if better
            if self.objective == "min":
                if score < self.best_score:
                    self.best_score = score
                    self.best_config = self._current_config.copy()
            else:  # "max"
                if score > self.best_score:
                    self.best_score = score
                    self.best_config = self._current_config.copy()
        
        # Generate next point
        if len(self.X) > 1:  # Need at least 2 points to fit GP
            self.gpr.fit(np.array(self.X), np.array(self.Y))
            next_config = self._predict_next()
        else:
            next_config = self._sample_random_point()
        
        # Store for evaluation in next call
        self._current_config = next_config
        self.current_iteration += 1
        
        # Check if we've reached the iteration limit
        if self.current_iteration >= self.max_iterations:
            self._stopped = True
            
        return next_config

    def _predict_next(self) -> pMap:
        """
        Uses GP to predict promising points.
        """
        cfg = self._algorithm.get_config()
        bds = self._algorithm.get_bounds()
        
        # Sample a set of random candidates
        num_candidates = 20
        candidates = []
        
        for _ in range(num_candidates):
            candidate = {}
            for key, (typ, _) in cfg.items():
                bounds = bds.get(key)
                if bounds is None:
                    continue
                    
                low, high = bounds
                if typ is float:
                    candidate[key] = random.uniform(low, high)
                elif typ is int:
                    candidate[key] = random.randint(int(low), int(high))
                elif typ is bool:
                    candidate[key] = random.choice([True, False])
                elif typ is list:
                    if isinstance(bounds, list):
                        candidate[key] = random.choice(bounds)
                    else:
                        candidate[key] = []
            candidates.append((list(candidate.values()), candidate))
            
        # Evaluate candidates with acquisition function
        best_val = float("inf") if self.objective == "min" else float("-inf")
        best_candidate = candidates[0][1] if candidates else self._sample_random_point()
        
        for values, cand_dict in candidates:
            x_candidate = np.array(values).reshape(1, -1)
            mean, std = self.gpr.predict(x_candidate, return_std=True)
            
            # Acquisition function (Expected Improvement)
            if self.objective == "min":
                acq_value = float(mean) - 0.1 * float(std)
                if acq_value < best_val:
                    best_val = acq_value
                    best_candidate = cand_dict
            else:  # maximization
                acq_value = float(mean) + 0.1 * float(std)
                if acq_value > best_val:
                    best_val = acq_value
                    best_candidate = cand_dict
                
        return best_candidate

    def _sample_random_point(self) -> pMap:
        """Helper to generate a random point within bounds."""
        cfg = self._algorithm.get_config()
        bds = self._algorithm.get_bounds()
        params = {}
        
        for key, (typ, _) in cfg.items():
            bounds = bds.get(key)
            if bounds is None:
                # NOTE: What do we do if the bound is not specified? Please realize that this will cause a silent failure that's hard to debug.
                # - by Dennis
                continue
                
            low, high = bounds
            if typ is float:
                params[key] = random.uniform(low, high)
            elif typ is int:
                params[key] = random.randint(int(low), int(high))
            elif typ is bool:
                params[key] = random.choice([True, False])
            elif typ is list:
                if isinstance(bounds, list):
                    params[key] = random.choice(bounds)
                else:
                    params[key] = []
        return params

    def is_finished(self) -> bool:
        """Determines if optimization is complete."""
        if self._stopped:
            return True

**Load event data**

In [16]:
events = []
n_files = 150

for i in range(0, n_files):
    if i == 51:
        """
        There's an issue with event 51 -> module_prefix_sum contains value 79 twice resulting in and indexing error when loading the event
        """
        print(f"Skipping problematic file: velo_event_{i}.json")
    else:    
        print(f"Loading file: velo_event_{i}.json")
        event_file = open(os.path.join("../DB/raw", f"velo_event_{i}.json"))
        json_data = json.loads(event_file.read())
        events.append(json_data) # type: ignore
        event_file.close()

Loading file: velo_event_0.json
Loading file: velo_event_1.json
Loading file: velo_event_2.json
Loading file: velo_event_3.json
Loading file: velo_event_4.json
Loading file: velo_event_5.json
Loading file: velo_event_6.json
Loading file: velo_event_7.json
Loading file: velo_event_8.json
Loading file: velo_event_9.json
Loading file: velo_event_10.json
Loading file: velo_event_11.json
Loading file: velo_event_12.json
Loading file: velo_event_13.json
Loading file: velo_event_14.json
Loading file: velo_event_15.json
Loading file: velo_event_16.json
Loading file: velo_event_17.json
Loading file: velo_event_18.json
Loading file: velo_event_19.json
Loading file: velo_event_20.json
Loading file: velo_event_21.json
Loading file: velo_event_22.json
Loading file: velo_event_23.json
Loading file: velo_event_24.json
Loading file: velo_event_25.json
Loading file: velo_event_26.json
Loading file: velo_event_27.json
Loading file: velo_event_28.json
Loading file: velo_event_29.json
Loading file: velo_e

In [17]:
pipeline = TrackFollowingPipeline(events=events, intra_node=True) # type: ignore 

In [18]:
Optimiser = Bayesian(learning_rate=0.1, max_iterations=243, target_score=0.001, objective="min") # type: ignore
optimal_parameters = pipeline.optimise_parameters(Optimiser, max_runs=2430) # DO NOT remove max_runs, chances are that this will run forever

Optimising:   0%|          | 2/2430 [00:02<49:19,  1.22s/it]/Users/svenhockers/Documents/code_files/velopix_tracking/venv/lib/python3.13/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/svenhockers/Documents/code_files/velopix_tracking/venv/lib/python3.13/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 0.0001. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/var/folders/cz/r9zhzycs23b9pwh_wffb3kt80000gn/T/ipykernel_9982/2222810077.py:132: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from 

Finsihed condition met, exiting...


In [ ]:
print(optimal_parameters) # Note these are just here for example...

{'x_slope': 0.8644562950913536, 'y_slope': 0.9043803161855442, 'x_tol': 0.7813277009782141, 'y_tol': 0.7724040842509098, 'scatter': 0.43846130911947284}
